<h1> Initial Data Handling

In [2]:
#IMPORTS

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from utils_preprocess_Apr_1 import make_dataframe

%matplotlib inline 

#GET DATA

numpy_seed = 999
np.random.seed(numpy_seed)

'''Prepare channel flow data'''
#Retau_train = [5200] #Use this to train only with one case
Retau_train1 = [1000]
Retau_train2 = [2000]
Retau_train3 = [5200]

df_train1 = pd.concat([make_dataframe(Retau, nondim='k-eps') for Retau in Retau_train1])
df_train2 = pd.concat([make_dataframe(Retau, nondim='k-eps') for Retau in Retau_train2])
df_train3 = pd.concat([make_dataframe(Retau, nondim='k-eps') for Retau in Retau_train3])
df_train1['a'] = df_train1[['a_uv', 'a_uu', 'a_vv', 'a_ww']].values.tolist()
df_train2['a'] = df_train2[['a_uv', 'a_uu', 'a_vv', 'a_ww']].values.tolist()
df_train3['a'] = df_train3[['a_uv', 'a_uu', 'a_vv', 'a_ww']].values.tolist()

Retau_test = [550]
df_test = pd.concat([make_dataframe(Retau, nondim='k-eps') for Retau in Retau_test])
df_test['a'] = df_test[['a_uv', 'a_uu', 'a_vv', 'a_ww']].values.tolist()

# scale up target variable in order for stable training, will consider implementing automatic scaler 
df_train1['a_uv'] *= 1e1
df_train2['a_uv'] *= 1e1   
df_train3['a_uv'] *= 1e1   
df_test['a_uv'] *= 1e1 
df_train1['Re_tau'] *= 1e-8 #-6 a -2
df_train2['Re_tau'] *= 1e-8   
df_train3['Re_tau'] *= 1e-8   
df_test['Re_tau'] *= 1e-8 
#Get validation data but can ignore it in this case
df_val=df_test

#Set random seed
torch_seed = 9
torch.manual_seed(torch_seed)

target_label = ['a_uv'] # or ['a'] (output_len = 1 if 'a_uv'; output_len = 4 if 'a')



KeyError: 1000

In [2]:
#CONVERT DATA TO THE RIGHT TYPE

import torch
from torch import nn

In [3]:
#Format training1 x
xtrain1=df_train1['du_dy'].values
xtrain_torch1=torch.from_numpy(xtrain1)
xtrain_in1=xtrain_torch1.view(1,1,len(df_train1)).float()
#Add Reynolds number
re_train1=df_train1['Re_tau'].values
re_train1=torch.from_numpy(re_train1)
re_train1=re_train1.view(1,1,len(re_train1)).float()
xtrain_in1=torch.cat((xtrain_in1,re_train1),dim=1)

#Format training1 y+ (for boundary condition enforcement)
yplus_train1=df_train1['y+'].values
yplus_train_torch1=torch.from_numpy(yplus_train1)
yplus_train_in1=yplus_train_torch1.view(1,1,len(df_train1)).float()

#Format training1 y
ytrain1=df_train1['a_uv'].values
ytrain_torch1=torch.from_numpy(ytrain1)
ytrain_in1=ytrain_torch1.view(1,1,len(df_train1)).float()

#Format training2 x
xtrain2=df_train2['du_dy'].values
xtrain_torch2=torch.from_numpy(xtrain2)
xtrain_in2=xtrain_torch2.view(1,1,len(df_train2)).float()
#Add Reynolds number
re_train2=df_train2['Re_tau'].values
re_train2=torch.from_numpy(re_train2)
re_train2=re_train2.view(1,1,len(re_train2)).float()
xtrain_in2=torch.cat((xtrain_in2,re_train2),dim=1)

#Format training2 y+ (for boundary condition enforcement)
yplus_train2=df_train2['y+'].values
yplus_train_torch2=torch.from_numpy(yplus_train2)
yplus_train_in2=yplus_train_torch2.view(1,1,len(df_train2)).float()

#Format training2 y
ytrain2=df_train2['a_uv'].values
ytrain_torch2=torch.from_numpy(ytrain2)
ytrain_in2=ytrain_torch2.view(1,1,len(df_train2)).float()

#Format training3 x
xtrain3=df_train3['du_dy'].values
xtrain_torch3=torch.from_numpy(xtrain3)
xtrain_in3=xtrain_torch3.view(1,1,len(df_train3)).float()
#Add Reynolds number
re_train3=df_train3['Re_tau'].values
re_train3=torch.from_numpy(re_train3)
re_train3=re_train3.view(1,1,len(re_train3)).float()
xtrain_in3=torch.cat((xtrain_in3,re_train3),dim=1)

#Format training3 y+ (for boundary condition enforcement)
yplus_train3=df_train3['y+'].values
yplus_train_torch3=torch.from_numpy(yplus_train3)
yplus_train_in3=yplus_train_torch3.view(1,1,len(df_train3)).float()

#Format training3 y
ytrain3=df_train3['a_uv'].values
ytrain_torch3=torch.from_numpy(ytrain3)
ytrain_in3=ytrain_torch3.view(1,1,len(df_train3)).float()

<h1> Input Data Formatting

In [4]:
#Format validation x
xval=df_val['du_dy'].values
xval_torch=torch.from_numpy(xval)
xval_in=xval_torch.view(1,1,len(df_val)).float()
#Add Reynolds number
re_trainval=df_val['Re_tau'].values
re_trainval=torch.from_numpy(re_trainval)
re_trainval=re_trainval.view(1,1,len(re_trainval)).float()
xval_in=torch.cat((xval_in,re_trainval),dim=1)

#Format validation y+ (for boundary condition enforcement)
yplus_val=df_val['y+'].values
yplus_val_torch=torch.from_numpy(yplus_val)
yplus_val_in=yplus_val_torch.view(1,1,len(df_val)).float()

#Format validation y
yval=df_val['a_uv'].values
yval_torch=torch.from_numpy(yval)
yval_in=yval_torch.view(1,1,len(df_val)).float()

#Format testing x
xtest=df_test['du_dy'].values
xtest_torch=torch.from_numpy(xtest)
xtest_in=xtest_torch.view(1,1,len(df_test)).float()
#Add Reynolds number
re_traintest=df_test['Re_tau'].values
re_traintest=torch.from_numpy(re_traintest)
re_traintest=re_traintest.view(1,1,len(re_traintest)).float()
xtest_in=torch.cat((xtest_in,re_traintest),dim=1)

#Format testing y+ (for boundary condition enforcement)
yplus_test=df_test['y+'].values
yplus_test_torch=torch.from_numpy(yplus_test)
yplus_test_in=yplus_test_torch.view(1,1,len(df_test)).float()

#Format testing y
ytest=df_test['a_uv'].values
ytest_torch=torch.from_numpy(ytest)
ytest_in=ytest_torch.view(1,1,len(df_test)).float()

In [5]:
maxlength=max(xtrain1.shape[0],xtrain2.shape[0],xtrain3.shape[0],xval.shape[0],xtest.shape[0])

In [6]:
shapex1=xtrain1.shape[0]
shapex2=xtrain2.shape[0]
shapex3=xtrain3.shape[0]
shapexval=xval.shape[0]
shapextest=xtest.shape[0]
zerostrain1=torch.zeros(1,1,maxlength-xtrain1.shape[0])
zerostrain2=torch.zeros(1,1,maxlength-xtrain2.shape[0])
zerostrain3=torch.zeros(1,1,maxlength-xtrain3.shape[0])
zerosval=torch.zeros(1,1,maxlength-xval.shape[0])
zerostest=torch.zeros(1,1,maxlength-xtest.shape[0])

zerostrain1_double=torch.zeros(1,2,maxlength-xtrain1.shape[0])
zerostrain2_double=torch.zeros(1,2,maxlength-xtrain2.shape[0])
zerostrain3_double=torch.zeros(1,2,maxlength-xtrain3.shape[0])
zerosval_double=torch.zeros(1,2,maxlength-xval.shape[0])
zerostest_double=torch.zeros(1,2,maxlength-xtest.shape[0])
# converting the data into GPU format
xtrain_in1=torch.cat((xtrain_in1,zerostrain1_double), 2)
yplus_train_in1=torch.cat((yplus_train_in1,zerostrain1), 2)
ytrain_in1=torch.cat((ytrain_in1,zerostrain1), 2)

xtrain_in2=torch.cat((xtrain_in2,zerostrain2_double), 2)
yplus_train_in2=torch.cat((yplus_train_in2,zerostrain2), 2)
ytrain_in2=torch.cat((ytrain_in2,zerostrain2), 2)

xtrain_in3=torch.cat((xtrain_in3,zerostrain3_double), 2)
yplus_train_in3=torch.cat((yplus_train_in3,zerostrain3), 2)
ytrain_in3=torch.cat((ytrain_in3,zerostrain3), 2)

xval_in=torch.cat((xval_in,zerosval_double), 2)
yplus_val_in=torch.cat((yplus_val_in,zerosval), 2)
yval_in=torch.cat((yval_in,zerosval), 2)

xtest_in=torch.cat((xtest_in,zerostest_double), 2)
yplus_test_in=torch.cat((yplus_test_in,zerostest), 2)
ytest_in=torch.cat((ytest_in,zerostest), 2)

<h1> Create the Convolutional Neural Network with Boundary Condition Enforcement

In [7]:
#CREATE THE MODEL INSIDE A CLASS
#import required for the activation functions
import torch.nn.functional as F

class CONV_NN_BC(nn.Module):
    def __init__(self, structure=None):
        super(CONV_NN_BC, self).__init__()

        self.fc0=nn.Sequential(
                nn.Conv1d(2, 5, 3, 1),
                nn.BatchNorm1d(5,affine=True),
                nn.ELU(inplace=True))
        self.fc1=nn.Sequential(
                nn.Conv1d(5, 5, 11, 1),
                nn.BatchNorm1d(5,affine=True),
                nn.ELU(inplace=True))
        self.fc2=nn.Sequential(
                nn.Conv1d(5, 10, 31, 1),
                nn.BatchNorm1d(10,affine=True),
                nn.ELU(inplace=True))
        self.fc3=nn.Sequential(
                nn.Conv1d(10, 10, 41, 1),
                nn.BatchNorm1d(10,affine=True),
                nn.ELU(inplace=True))
        self.fc4=nn.Sequential(
                nn.Conv1d(10, 10, 41, 1),
                #nn.BatchNorm1d(10),
                nn.ELU(inplace=True))
        
        
        self.final = nn.Conv1d(1,1, 10, stride=10)
        self.f = nn.Flatten()
        self.zerotensor0=torch.zeros(1,2,1)
        self.zerotensor1=torch.zeros(1,5,5)
        self.zerotensor2=torch.zeros(1,5,15)
        self.zerotensor3=torch.zeros(1,10,20)
        self.zerotensor4=torch.zeros(1,10,20)
        self.beta=0.1

        
    def forward(self,x,yplus):
        re = x[:,1,:]
        x = torch.cat((x,self.zerotensor0), 2)
        x = torch.cat((self.zerotensor0,x), 2)
        x = self.fc0(x)
        #x = torch.cat((x,re),1)
        #pad tensor
        x = torch.cat((x,self.zerotensor1), 2)
        x = torch.cat((self.zerotensor1,x), 2)
        #Convolution
        x = self.fc1(x)
        #pad tensor
        x = torch.cat((x,self.zerotensor2), 2)
        x = torch.cat((self.zerotensor2,x), 2)
        #Convolution
        x = self.fc2(x)
        #pad tensor
        x = torch.cat((x,self.zerotensor3), 2)
        x = torch.cat((self.zerotensor3,x), 2)
        #Convolution
        x = self.fc3(x)
        #pad tensor
        x = torch.cat((x,self.zerotensor4), 2)
        x = torch.cat((self.zerotensor4,x), 2)
        #Convolution
        x = self.fc4(x)
        #Final layer 
        x = torch.mean(x,dim=0) #reshape tensor to apply transpose
        x = torch.transpose(x, 0, 1) #transpose tensor
        x = x[None,:,:] #reshape tensor
        x = self.f(x) #Flatten
        x = x[None,:,:] #reshape tensor
        x = self.final(x) #weighted average (set initial weights to 1 and divide by 8)
        #output = x
        output = (1. - torch.exp(-self.beta * yplus))*x

        return output

In [8]:
#define new loss function

#def my_loss(output, target,y):
    #loss = torch.mean(((output - target)**2)/(y+400))
#    loss = torch.mean((output - target)**2)
#    loss
#    return loss

In [9]:
#CALL MODEL AND DEFINE LOSS AND OPTIMIZER FOR THE FUNCTION
# defining the model
model = CONV_NN_BC()

#CHOOSE CASE (remember to specify case in the first cell Retau_test)
#model = torch.load('short_convbcre_first') #for test 5200
#model = torch.load('short_convbcre_second') #for test 2000
#model = torch.load('short_convbcre_third') #for test 1000
model = torch.load('short_convbcre_fourth') #for test 550

In [10]:
criterion = nn.MSELoss()
xtrain_in1=xtrain_in1[:,:,0:shapex1]
xtrain_in2=xtrain_in2[:,:,0:shapex2]
xtrain_in3=xtrain_in3[:,:,0:shapex3]
yplus_train_in1=yplus_train_in1[:,:,0:shapex1]
yplus_train_in2=yplus_train_in2[:,:,0:shapex2]
yplus_train_in3=yplus_train_in3[:,:,0:shapex3]
ytrain_in1=ytrain_in1[:,:,0:shapex1]
ytrain_in2=ytrain_in2[:,:,0:shapex2]
ytrain_in3=ytrain_in3[:,:,0:shapex3]


xval_in=xval_in[:,:,0:shapexval]
yplus_val_in=yplus_val_in[:,:,0:shapexval]
yval_in=yval_in[:,:,0:shapexval]

xtest_in=xtest_in[:,:,0:shapextest]
yplus_test_in=yplus_test_in[:,:,0:shapextest]
ytest_in=ytest_in[:,:,0:shapextest]

a_train1=model(xtrain_in1,yplus_train_in1)
a_train2=model(xtrain_in2,yplus_train_in2)
a_train3=model(xtrain_in3,yplus_train_in3)
a_train=torch.cat((a_train1,a_train2,a_train3),2)
a_train=torch.mean(a_train, dim=0)
a_train=torch.mean(a_train, dim=0)
b_train=torch.cat((ytrain_in1,ytrain_in2,ytrain_in3),2)
b_train=torch.mean(b_train, dim=0)
b_train=torch.mean(b_train, dim=0)

a_val=model(xval_in,yplus_val_in)
a_val=torch.mean(a_val, dim=0)
a_val=torch.mean(a_val, dim=0)
b_val=torch.mean(yval_in, dim=1)
b_val=torch.mean(b_val, dim=0)
test_performance=criterion(ytest_in,model(xtest_in,yplus_test_in))
a_test=torch.mean(model(xtest_in,yplus_test_in), dim=0)
a_test=torch.mean(a_test, dim=0)
b_test=torch.mean(ytest_in, dim=1)
b_test=torch.mean(b_test, dim=0)
a_train=a_train.cpu().detach().numpy() 
b_train=b_train.cpu().detach().numpy()
a_val=a_val.cpu().detach().numpy() 
b_val=b_val.cpu().detach().numpy()
a_test=a_test.cpu().detach().numpy() 
b_test=b_test.cpu().detach().numpy()


from sklearn.metrics import r2_score
print('R^2: ','\t', 'train:', r2_score(b_train,a_train), 'val:', r2_score(b_val,a_val), 'test:', r2_score(b_test,a_test))



test_performance_piecewise=np.zeros_like(a_test)
for i in range(len(a_test)):
####################################################################################   
    test_performance_piecewise[i]=criterion(ytest_in[0,0,i],model(xtest_in,yplus_test_in)[0,0,i])
####################################################################################

R^2:  	 train: 0.9999472030742749 val: 0.9949096453250155 test: 0.9949096453250155
